In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[3],3a
commitnumber,1013f91,5014a19
conc,-,3.2e-07
dv,-,0.001
klin,-,2.22e-20
molecule,{'n2o': 3.2e-07},n2o
ng_adju,-,"[0, 0]"
ng_refs,-,"[1, 2]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.001855                    1  0.006638
0.000750        2  0.002047                    2  0.006790
0.001052        3  0.002444                    3  0.007119
0.001476        4  0.002849                    4  0.007446
0.002070        5  0.003258                    5  0.007778
0.002904        6  0.003668                    6  0.008109
0.004074        7  0.004080                    7  0.008447
0.005714        8  0.004494                    8  0.008786
0.008015        9  0.004907                    9  0.009125
0.011243       10  0.005322                   10  0.009469
0.015771       11  0.005715                   11  0.009811
0.022122       12  0.006092                   12  0.010156
0.031031       13  0.006444                   13  0.010502
0.043528       14  0.006890                   14  0.010950
0.061057       15  0.008756                   15  0.012567
0.085645       16  0.012771                   16  0.015943
0.120136       17  0.017812                   17  0.020193
0.168516       18  0.023229                   18  0.024801
0.236378       19  0.029012                   19  0.029762
0.331549       20  0.035167                   20  0.035079
0.465100       21  0.041694                   21  0.040768
0.652400       22  0.047892                   22  0.046217
0.915100       23  0.049528                   23  0.048041
1.283650       24  0.046452                   24  0.046056
1.800600       25  0.041716                   25  0.042576
2.525700       26  0.035857                   26  0.037898
3.542800       27  0.030188                   27  0.032963
4.969550       28  0.025118                   28  0.028084
6.970850       29  0.020616                   29  0.023204
9.778100       30  0.016569                   30  0.018330
13.715850      31  0.012803                   31  0.013428
19.239350      32  0.009138                   32  0.008565
26.987250      33  0.005425                   33  0.003814
37.855300      34  0.001487                   34 -0.000786
53.100050      35 -0.003774                   35 -0.005923
73.887500      36 -0.010115                   36 -0.011297
97.662500      37 -0.014579                   37 -0.014763
121.437500     38 -0.014752                   38 -0.014770
145.212500     39 -0.012417                   39 -0.012776
168.987500     40 -0.010324                   40 -0.010835
192.762500     41 -0.008392                   41 -0.008957
216.537500     42 -0.006572                   42 -0.007136
240.312500     43 -0.004832                   43 -0.005363
264.087500     44 -0.003163                   44 -0.003635
287.862500     45 -0.001553                   45 -0.001950
311.637500     46  0.000006                   46 -0.000304
335.412500     47  0.001520                   47  0.001301
359.187500     48  0.002994                   48  0.002871
382.962500     49  0.004431                   49  0.004407
406.737500     50  0.005836                   50  0.005910
430.512500     51  0.007211                   51  0.007381
454.287500     52  0.008560                   52  0.008822
478.062500     53  0.009885                   53  0.010239
501.837500     54  0.011189                   54  0.011630
525.612500     55  0.012473                   55  0.012991
549.387500     56  0.013741                   56  0.014328
573.162500     57  0.014995                   57  0.015646
596.937500     58  0.016235                   58  0.016943
620.712500     59  0.017458                   59  0.018220
644.487500     60  0.018660                   60  0.019464
668.262500     61  0.019816                   61  0.020658
692.037500     62  0.020873                   62  0.021760
715.812500     63  0.021757                   63  0.022715
739.587500     64  0.022452                   64  0.023514
763.362500     65  0.023035                   65  0.024214
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -37.085129  0.000000 -37.085129 -37.140506  4.382719e-08   
0.000624    2     -37.085130  0.000001 -37.085128 -37.140507  1.149295e-06   
0.000876    3     -37.085130  0.000002 -37.085128 -37.140507  1.597217e-06   
0.001229    4     -37.085131  0.000003 -37.085128 -37.140507  2.232489e-06   
0.001723    5     -37.085132  0.000004 -37.085128 -37.140508  3.133433e-06   
0.002417    6     -37.085133  0.000005 -37.085128 -37.140509  4.411014e-06   
0.003391    7     -37.085134  0.000007 -37.085127 -37.140509  6.222704e-06   
0.004757    8     -37.085138  0.000011 -37.085126 -37.140511  8.792130e-06   
0.006672    9     -37.085140  0.000015 -37.085125 -37.140512  1.243580e-05   
0.009359    10    -37.085146  0.000021 -37.085124 -37.140515  1.760254e-05   
0.013128    11    -37.085151  0.000030 -37.085122 -37.140518  2.492933e-05   
0.018415    12    -37.085161  0.000043 -37.085118 -37.140522  3.531698e-05   
0.025830    13    -37.085173  0.000060 -37.085113 -37.140528  5.004434e-05   
0.036232    14    -37.085190  0.000086 -37.085105 -37.140536  7.092281e-05   
0.050823    15    -37.085214  0.000121 -37.085093 -37.140546  1.006124e-04   
0.071291    16    -37.085245  0.000174 -37.085072 -37.140560  1.442924e-04   
0.100000    17    -37.085282  0.000254 -37.085028 -37.140573  2.115028e-04   
0.140271    18    -37.085322  0.000378 -37.084943 -37.140581  3.162581e-04   
0.196760    19    -37.085358  0.000571 -37.084788 -37.140578  4.791269e-04   
0.275997    20    -37.085382  0.000867 -37.084515 -37.140551  7.315909e-04   
0.387100    21    -37.085373  0.001320 -37.084053 -37.140479  1.121541e-03   
0.543100    22    -37.085293  0.002011 -37.083282 -37.140327  1.722974e-03   
0.761700    23    -37.085088  0.003047 -37.082042 -37.140045  2.637864e-03   
1.068500    24    -37.084747  0.004505 -37.080242 -37.139616  3.954845e-03   
1.498800    25    -37.084300  0.006425 -37.077874 -37.139061  5.747602e-03   
2.102400    26    -37.083770  0.008879 -37.074891 -37.138399  8.130056e-03   
2.949000    27    -37.083223  0.011927 -37.071295 -37.137689  1.122140e-02   
4.136600    28    -37.082734  0.015685 -37.067049 -37.137015  1.518489e-02   
5.802500    29    -37.082420  0.020327 -37.062092 -37.136510  2.022233e-02   
8.139200    30    -37.082487  0.026101 -37.056386 -37.136417  2.655343e-02   
11.417000   31    -37.083301  0.033349 -37.049952 -37.137155  3.440915e-02   
16.014700   32    -37.085489  0.042510 -37.042979 -37.139442  4.400947e-02   
22.464000   33    -37.090110  0.054112 -37.035998 -37.144463  5.557541e-02   
31.510500   34    -37.098908  0.068724 -37.030184 -37.154167  6.936661e-02   
44.200100   35    -37.114819  0.086870 -37.027948 -37.171697  8.571534e-02   
62.000000   36    -37.143847  0.107941 -37.035906 -37.202837  1.043658e-01   
85.775000   37    -37.193960  0.129567 -37.064393 -37.254182  1.238925e-01   
109.550000  38    -37.253467  0.148015 -37.105453 -37.312833  1.409613e-01   
133.325000  39    -37.316385  0.169388 -37.146997 -37.373761  1.602877e-01   
157.100000  40    -37.378537  0.196570 -37.181967 -37.433853  1.843944e-01   
180.875000  41    -37.439573  0.228529 -37.211044 -37.492847  2.128703e-01   
204.650000  42    -37.499288  0.264610 -37.234680 -37.550579  2.453742e-01   
228.425000  43    -37.557545  0.304356 -37.253188 -37.606923  2.816179e-01   
252.200000  44    -37.614188  0.347391 -37.266797 -37.661769  3.213602e-01   
275.975000  45    -37.669144  0.393441 -37.275704 -37.715024  3.643766e-01   
299.750000  46    -37.722370  0.442294 -37.280076 -37.766615  4.104760e-01   
323.525000  47    -37.773830  0.493771 -37.280059 -37.816493  4.594977e-01   
347.300000  48    -37.823501  0.547721 -37.275780 -37.864603  5.112713e-01   
371.075000  49    -37.871364  0.604015 -37.2673

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')